# Ensemble model

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
sys.path.append("../src")
from ensemble import EnsembleModel

d:\anaconda3\envs\cs4248\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, RobertaTokenizerFast

## Step 1: Load models

### Distilbert

In [5]:
distilbert_name = "jeffnjy/distilbert-base-test"
distilbert_tokenizer = AutoTokenizer.from_pretrained(distilbert_name)
distilbert_model = AutoModelForQuestionAnswering.from_pretrained(distilbert_name)

### Albert

In [6]:
albert_name = "jeffnjy/albert-base-test"
albert_tokenizer = AutoTokenizer.from_pretrained(albert_name)
albert_model = AutoModelForQuestionAnswering.from_pretrained(albert_name)

### Roberta

In [7]:
roberta_name = "jeffnjy/roberta-base-test"
roberta_tokenizer = RobertaTokenizerFast.from_pretrained(roberta_name)
roberta_model = AutoModelForQuestionAnswering.from_pretrained(roberta_name)

### Bert

In [8]:
bert_name = "jeffnjy/bert-base-test"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_name)
bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_name)

d:\anaconda3\envs\cs4248\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\px109\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


### XLNet

In [9]:
xlnet_name = "JiayanL/XLNET"
xlnet_tokenizer = AutoTokenizer.from_pretrained(xlnet_name)
xlnet_model = AutoModelForQuestionAnswering.from_pretrained(xlnet_name)

## Step 2: Create ensemble model

The weight of each model can be changed here

In [10]:
model_weights = {
    "bert": 0.2,
    "albert": 0.4,
    "distilbert": 0.3,
    "roberta": 1.0,
    "xlnet": 0.5,
}
mode = "soft" # soft/hard
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [47]:
ensemble_model = EnsembleModel(device=device)
ensemble_model.add_model(model=bert_model, tokenizer=bert_tokenizer, weight=model_weights["bert"])
ensemble_model.add_model(model=albert_model, tokenizer=albert_tokenizer, weight=model_weights["albert"])
ensemble_model.add_model(model=distilbert_model, tokenizer=distilbert_tokenizer, weight=model_weights["distilbert"])
ensemble_model.add_model(model=roberta_model, tokenizer=roberta_tokenizer, weight=model_weights["roberta"])
ensemble_model.add_model(model=xlnet_model, tokenizer=xlnet_tokenizer, weight=model_weights["xlnet"])

## Step 3: Perform perdiction

### 3.a Perform perdiction on one example

In [21]:
question = "Which NFL team represented the AFC at Super Bowl 50?"
context = """Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) 
for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football 
Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 
7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the 
league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the 
tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl 
L"), so that the logo could prominently feature the Arabic numerals 50."""

#### Option 1: Weighted sum of softmax probabilities (mode="soft")

In [48]:
ensemble_model.predict(question=question, context=context, mode="soft")

'Denver Broncos'

#### Option 2: Weighted votes (mode="hard")

In [36]:
ensemble_model.predict(question=question, context=context, mode="hard")

'Denver Broncos'

### 3.b Perform perdiction on a json dataset and generate a perdiction json file

In [15]:
inpath = "../dataset/dev-v1.1.json"
outpath = "../tmp/prediction/ensemble_all_exact.json"
ensemble_model.generate_prediction_json(inpath, outpath, mode)

100%|██████████| 10570/10570 [36:30<00:00,  4.83it/s] 


### 3.c Weight search

In [18]:
search_params = [
    (0.1, 0.4, 0.2, 1.0, 0.8, "soft"),
    (0.0, 0.4, 0.0, 1.0, 0.8, "soft"),
    (0.0, 0.0, 0.0, 1.0, 0.0, "soft"),
]

In [19]:
inpath = "../dataset/dev-v1.1.json"
for param in search_params:
    x, y, z, w, u, m = param
    ensemble_model = EnsembleModel(device=device)
    ensemble_model.add_model(model=bert_model, tokenizer=bert_tokenizer, weight=x)
    ensemble_model.add_model(model=albert_model, tokenizer=albert_tokenizer, weight=y)
    ensemble_model.add_model(model=distilbert_model, tokenizer=distilbert_tokenizer, weight=z)
    ensemble_model.add_model(model=roberta_model, tokenizer=roberta_tokenizer, weight=w)
    ensemble_model.add_model(model=xlnet_model, tokenizer=xlnet_tokenizer, weight=u)
    outpath = f"../tmp/prediction/ensemble_{x}_{y}_{z}_{w}_{u}_{m}.json"
    ensemble_model.generate_prediction_json(inpath, outpath, m)

100%|██████████| 10570/10570 [34:34<00:00,  5.09it/s]
